# Neural Network for XoR function in Python
## En el tutorial 2 habimos creado una pequeña red que intentaba aprender la función XoR, pero nos dimos cuenta que mismo despues de 10.000 iteraciones nuestro error no lograba disminuir, incluso aumentaba.
### En este laboratorio iremos a jugar con las posibles causas para poder descobrir como solucionar el problema

### Empecemos entendendo el código

In [1]:
import numpy as np
import random

class Sigmoid:
    def derivative(self, x):
        return x * (1 - x)
    
    def __call__(self, x):
        return 1 / (1 + np.exp(-x))
sigmoid = Sigmoid()

### En la celda de arriba tenemos nuestra función de activación y su derivada

In [2]:
# Inicialización aleatoria con valores entre -1 y 1
w11, w21, w12, w22, w_n1, w_n2 = 2*np.random.random() - 1, 2*np.random.random() - 1, 2*np.random.random() - 1, 2*np.random.random() - 1, 2*np.random.random() - 1, 2*np.random.random() - 1
b1, b2, b3 = 2*np.random.random() - 1, 2*np.random.random() - 1, 2*np.random.random() - 1

# Datos XOR
entradas = np.array([[0,0], [0,1], [1,0], [1,1]])
salidas_deseadas = np.array([[0], [1], [1], [0]])

### Acá estavamos inicializando los pesos y sesgos, además de definir nuestro valores de entrar, empezemos haciendo una pequeña modificación, para que nuestro código pueda ser reutilizable!

In [3]:
class Neuron:
    def __init__(self, weights, bias, activation = sigmoid):
        self.weights = weights
        self.bias = bias
        self.activation = activation
        self.output = 0
        self.inputs = []
        self.error = 0

    def feedforward(self, inputs):
        self.inputs = inputs
        total = np.dot(self.weights, inputs) + self.bias
        self.output = self.activation(total)
        return self.output
    
    def backpropagate_error(self, error):
        self.error = self.activation.derivative(self.output) * error

### Ahora tenemos una clase Nuerona, esta nos va servir para guardar la información de su valores de entrada, salida y además
### va facilitar la propagación de error, tema que no habiamos visto en la tutoria 2

### Tome su tiempo para leer y entender el código 

In [4]:
class NeuralNetwork:
    def __init__(self, input_red, output, h_layers, learning_rate):
        self.learning_rate = 0.1
        self.layers = []
        self.input = input_red
        self.__init__layers(h_layers, output)
    
    def __init__layers(self, h_layers, output):
        for indice_layer in range(len(h_layers)):
            
                layer = []
                for cantidad_de_neuronas in range(h_layers[indice_layer]):
                    if indice_layer == 0:
                        neurona = Neuron(
                            [random.uniform(-1, 1) for _ in range(self.input)], random.uniform(-1, 1)
                        )
                    else:
                        neurona = Neuron(
                            [random.uniform(-1, 1) for _ in range(h_layers[indice_layer-1])], random.uniform(-1, 1)
                        )
                    layer.append(neurona)
                self.layers.append(layer)
                
        self.layers.append([Neuron([random.uniform(-1, 1) for _ in range(h_layers[-1])], random.uniform(-1, 1), sigmoid) for _ in range(output)])
    
    
    def feedforward(self, inputs):
        for indice_capas in range(len(self.layers)):
            outputs = []
            for indice_neurona in range(len(self.layers[indice_capas])):
                outputs.append(self.layers[indice_capas][indice_neurona].feedforward(inputs))
            inputs = outputs # para la proxima capa recibir lo que retorna la capa que acabamos de pasar
        return outputs
    
    # Acá era nuestro ciclo for para poder entrenar, pero encapsulado en un método
    def train(self, inputs, targets):
        outputs = self.feedforward(inputs)
        errors = [targets[i] - outputs[i] for i in range(len(outputs))]
    
        for indice_layers in range(len(self.layers)-1, -1, -1):
            layer = self.layers[indice_layers]
            next_layer = self.layers[indice_layers+1] if indice_layers != len(self.layers)-1 else None
            for indice_neurona in range(len(layer)):
                neuron = layer[indice_neurona]
                if next_layer is None:
                    neuron.backpropagate_error(errors[indice_neurona])
                else:
                    error = 0
                    for n in next_layer:
                        error += n.error * n.weights[indice_neurona]
                    neuron.backpropagate_error(error)    

        for indice_layers in range(len(self.layers)):
            for indice_neurona in range(len(self.layers[indice_layers])):
                
                neuron = self.layers[indice_layers][indice_neurona]
            
                for indice_peso in range(len(neuron.weights)):
                    neuron.weights[indice_peso] += self.learning_rate * neuron.error * neuron.inputs[indice_peso]
                neuron.bias += self.learning_rate * neuron.error

        error_final = sum([0.5*e**2 for e in errors]) # 1/2 Sum (i=0, n) [ y- ^y]**2
        return error_final

### Apesar del código ser mucho más largo, hace exactamente el mismo proceso que hemos estado haciendo, con la pequeña diferencia que estamos usando el algoritimo de backpropagate para poder calcular las derivadas de forma optima, de esta forma puedemos tener capas ocultas con varias y varias neuronas, y no tendremos que estar a mano calculando sus derivadas, el programa va hacer por nosotros =D

### Si antes ya era importante entender el código, ahora es más aún, así que leean con calma =)

In [5]:
nn = NeuralNetwork(2, 1, [2], learning_rate=0.0001)
#  input_red = 2, output = 1, h_layers= 1: misma configuración que teniamos antes
# h_layers = hidden layers o capas ocultas en español
    # XOR Input and Output
X = [[0, 0], [0, 1], [1, 0], [1, 1]]
Y = [[0], [1], [1], [0]]

# Training
for epoca in range(10000):
    error = 0
    for indice_error in range(len(X)):
        error += nn.train(X[indice_error], Y[indice_error])
    if epoca % 1000 == 0:
        print(f"Epoch {epoca} Error: {error}")

# Testing
for x in X:
    print(f"Input: {x}, Output: {nn.feedforward(x)}")

Epoch 0 Error: 0.520086646125137
Epoch 1000 Error: 0.503589305167198
Epoch 2000 Error: 0.49912747335043595
Epoch 3000 Error: 0.45942085061043914
Epoch 4000 Error: 0.38476100164178195
Epoch 5000 Error: 0.3291460369005969
Epoch 6000 Error: 0.090357676217333
Epoch 7000 Error: 0.028191727278672415
Epoch 8000 Error: 0.015302532717648252
Epoch 9000 Error: 0.010251606751514092
Input: [0, 0], Output: [0.06236281954183819]
Input: [0, 1], Output: [0.9262631029699286]
Input: [1, 0], Output: [0.9455774054154295]
Input: [1, 1], Output: [0.05436543594140357]


## Como ya habiamos visto en este ejemplo, nuestro error esta bien alto y no logra disminuir de 0.49, porque no estamos podiendo aprender la representación de la función XoR?
- Podemos cambiar algunos parametros para ver si el problema es nuestro learning rate? Sera la función de activación? O sera que la cantidad de neuronas que tenemos es demasiado pequeña para poder representar la función XoR?

# El objetivo es que el lector, consiga solucionar el problema, así que pasos a intentar
- aumentar learnig rate y ver si cambia el resultado
- aumentar el número de capas ocultas
- aumentar el número de epocas
- volver a disminuir el learning rate y dejar las capas ocultas en 2, que pasa ahora?